In [8]:
!pip install datasets

In [12]:
# Import required libraries
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd
from sklearn.metrics import accuracy_score

# Sample dataset with mental health-related text and sentiment labels
data = [
    ("I've been feeling quite good lately, enjoying my hobbies and spending time with friends.", "Normal"),
    ("I don't feel like doing anything anymore. Everything seems pointless.", "Depressed"),
    ("I'm feeling pretty stable these days, not much bothers me.", "Normal"),
    ("Waking up in the morning is a struggle. I feel drained and empty.", "Depressed"),
    ("I find pleasure in small things and feel content with my life.", "Normal"),
    ("Nothing interests me anymore, and I can't shake this feeling of sadness.", "Depressed"),
    ("I've been productive and focused, achieving my goals without much stress.", "Normal"),
    ("Every day feels the same, and I just want to disappear.", "Depressed"),
    ("I feel connected with my loved ones and enjoy our time together.", "Normal"),
    ("It's hard to even get out of bed; everything feels heavy and overwhelming.", "Depressed"),
    ("My mood has been stable, and I'm coping well with daily challenges.", "Normal"),
    ("I don't see the point in talking to anyone. I feel so isolated and alone.", "Depressed"),
    ("I've been laughing more and feeling optimistic about the future.", "Normal"),
    ("Nothing makes me happy anymore; I feel numb.", "Depressed"),
    ("I'm at peace with where I am in life and look forward to what's next.", "Normal"),
    ("I cry for no reason, and the sadness won't go away.", "Depressed"),
    ("I feel energized and motivated to take on new projects.", "Normal"),
    ("It's like a dark cloud is hanging over me, and I can't escape it.", "Depressed"),
    ("I've been feeling emotionally balanced and content.", "Normal"),
    ("I don't care about anything anymore. Life just feels so bleak.", "Depressed"),
    ("I'm happy with my life and the progress I'm making.", "Normal"),
    ("The sadness never leaves, and I feel hopeless all the time.", "Depressed"),
    ("I've been enjoying my work and feeling satisfied with my achievements.", "Normal"),
    ("I feel like I'm drowning in despair and can't find a way out.", "Depressed"),
    ("I'm feeling well and enjoying time with my family.", "Normal"),
    ("I don't see a future for myself; everything seems so dark.", "Depressed"),
    ("I'm able to relax and enjoy my free time.", "Normal"),
    ("It's hard to even smile; I feel so down and out.", "Depressed"),
    ("I've been feeling calm and at ease with my life.", "Normal"),
    ("The weight of my thoughts is crushing me, and I feel so low.", "Depressed"),
    ("I'm looking forward to the weekend and spending time outdoors.", "Normal"),
    ("I can't find joy in anything; I feel so empty inside.", "Depressed"),
    ("I've been feeling good about my relationships and personal growth.", "Normal"),
    ("The sadness is overwhelming, and I can't see a way through it.", "Depressed"),
    ("I feel relaxed and content with how things are going.", "Normal"),
    ("I can't stop thinking about how worthless I feel.", "Depressed"),
    ("I've been in a positive mood and enjoying life's little moments.", "Normal"),
    ("I feel so detached from everything, like I'm just going through the motions.", "Depressed"),
    ("I've been handling stress well and staying optimistic.", "Normal"),
    ("It's like a constant fog of sadness that won't lift.", "Depressed"),
    ("I'm enjoying my hobbies and finding fulfillment in my daily routine.", "Normal"),
    ("I feel so lost and hopeless, like there's no point in anything.", "Depressed"),
    ("I've been feeling happy and content with my progress in life.", "Normal"),
    ("I don't want to be around people; I just want to be alone in my sadness.", "Depressed"),
    ("I'm in a good place emotionally, feeling positive and hopeful.", "Normal"),
    ("Everything feels meaningless, and I can't escape the sadness.", "Depressed"),
    ("I've been feeling good about myself and my future.", "Normal"),
    ("I can't find the strength to face the day; everything feels so dark.", "Depressed"),
    ("I'm at peace with my life and enjoying my time with loved ones.", "Normal"),
    ("I feel like I'm just existing, not really living.", "Depressed"),
    ("I've been feeling upbeat and looking forward to new experiences.", "Normal"),
    ("The sadness never leaves, and I feel like I'm stuck in a dark hole.", "Depressed"),
    ("I'm enjoying my life and feel grateful for what I have.", "Normal"),
    ("I can't seem to escape these negative thoughts; they consume me.", "Depressed"),
    ("I'm feeling positive about my future and the direction I'm heading.", "Normal"),
    ("It's hard to get through the day without feeling completely overwhelmed.", "Depressed"),
    ("I'm feeling good about where I am and the progress I'm making.", "Normal"),
    ("I feel so disconnected from the world around me.", "Depressed"),
    ("I've been happy and content with my work-life balance.", "Normal"),
    ("Nothing brings me joy anymore, and I feel so empty.", "Depressed"),
    ("I'm satisfied with how things are going in my life.", "Normal"),
    ("The sadness is relentless, and I can't seem to shake it off.", "Depressed"),
    ("I've been feeling good about my health and overall well-being.", "Normal"),
    ("I feel like I'm trapped in my own mind, and I can't get out.", "Depressed"),
    ("I've been content with my life and enjoying my daily routine.", "Normal"),
    ("Everything feels so heavy, like I'm carrying the weight of the world.", "Depressed"),
    ("I'm in a good mood and feeling positive about the future.", "Normal"),
    ("The sadness is overwhelming, and I feel so alone.", "Depressed"),
    ("I've been feeling emotionally stable and content with my life.", "Normal"),
    ("I feel like I'm drowning in a sea of sadness, and there's no way out.", "Depressed"),
    ("I've been feeling good and enjoying the little things in life.", "Normal"),
    ("I feel so hopeless and can't find a way to move forward.", "Depressed"),
    ("I'm feeling positive about the direction my life is taking.", "Normal"),
    ("The sadness is always there, lurking in the background.", "Depressed"),
    ("I've been feeling well and content with where I am in life.", "Normal"),
    ("I feel like I'm just going through the motions, not really living.", "Depressed"),
    ("I'm happy with how things are going in my life and feel at peace.", "Normal"),
    ("The sadness is overwhelming, and I can't escape it.", "Depressed"),
    ("I've been feeling optimistic about the future and excited about new opportunities.", "Normal"),
    ("I feel so empty inside, like nothing matters anymore.", "Depressed"),
    ("I'm satisfied with how things are going and feeling positive about the future.", "Normal"),
    ("The sadness is suffocating, and I can't breathe.", "Depressed"),
    ("I've been feeling happy and content with my relationships and personal growth.", "Normal"),
    ("I feel like I'm stuck in a dark place, and I can't find a way out.", "Depressed"),
    ("I've been feeling good about my life and the direction I'm heading.", "Normal"),
    ("The sadness is overwhelming, and I can't escape this darkness.", "Depressed"),
    ("I'm feeling positive and looking forward to the future.", "Normal"),
    ("I feel so lost and alone, like no one understands me.", "Depressed"),
    ("I've been feeling content with my life and the progress I've made.", "Normal"),
    ("The sadness never leaves, and I can't see a way out.", "Depressed"),
    ("I've been enjoying life and feeling good about my future.", "Normal"),
    ("I feel like I'm drowning in sadness, and there's no one to save me.", "Depressed"),
    ("I'm happy with where I am in life and looking forward to new challenges.", "Normal"),
    ("The sadness is overwhelming, and I can't escape it.", "Depressed"),
    ("I've been feeling positive and excited about the future.", "Normal"),
    ("I feel like I'm trapped in a never-ending cycle of sadness.", "Depressed")
]

# Convert data into a pandas DataFrame
df = pd.DataFrame(data, columns=['text', 'label'])

# Encode labels (Depressed = 0, Normal = 1)
label_mapping = {'Depressed': 0, 'Normal': 1}
df['label_encoded'] = df['label'].map(label_mapping)

# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'].tolist(), df['label_encoded'].tolist(), test_size=0.2, random_state=42
)

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Tokenize the text data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

# Convert the data into Hugging Face Dataset format
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels
})

# Define a compute_metrics function to calculate accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    return {"accuracy": accuracy}

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',           # Output directory
    num_train_epochs=3,               # Number of training epochs
    per_device_train_batch_size=16,   # Batch size for training
    per_device_eval_batch_size=16,    # Batch size for evaluation
    evaluation_strategy="epoch",      # Evaluation strategy to evaluate at each epoch
    logging_dir='./logs',             # Directory for storing logs
    logging_steps=10,                 # Log every 10 steps
)

# Create a Trainer instance for training and evaluation
trainer = Trainer(
    model=model,                         # The pre-trained BERT model
    args=training_args,                  # Training arguments
    train_dataset=train_dataset,         # The training dataset
    eval_dataset=test_dataset,           # The test dataset for evaluation
    compute_metrics=compute_metrics      # Add the compute_metrics function
)

# Fine-tune the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Test Accuracy: {eval_results['eval_loss']:.2f}, Accuracy: {eval_results['eval_accuracy']:.2f}")

# Predict sentiment for a new text
new_text = "I feel really sad lately."
inputs = tokenizer(new_text, return_tensors="pt", truncation=True, padding=True, max_length=128)
outputs = model(**inputs)
predicted_class = torch.argmax(outputs.logits).item()
predicted_label = list(label_mapping.keys())[list(label_mapping.values()).index(predicted_class)]
print(f"Predicted sentiment: {predicted_label}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.476941,1.000000
2,0.584400,0.276648,1.000000
3,0.584400,0.208196,1.000000


Test Accuracy: 0.21, Accuracy: 1.00
Predicted sentiment: Normal
